In [37]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [47]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/10_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})
df = df.iloc[:1170]
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m')
df.set_index('Date', inplace=True)
df

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.13
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.35
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.84
1926-11-01,5.20,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11
...,...,...,...,...,...,...,...,...,...,...
2023-08-01,-3.77,-4.31,-2.37,1.95,-1.68,0.14,-0.40,-0.22,-5.29,-3.35
2023-09-01,-4.57,-2.58,-7.08,3.17,-5.98,-3.22,-5.68,-4.71,-5.04,-3.41
2023-10-01,-3.53,-17.88,-2.76,-6.24,-1.78,-0.18,0.47,-4.58,1.12,-2.53


In [12]:
window_size = 60
rolling_window = df.rolling(window=window_size)

In [29]:
# Calculer la moyenne des rendements pour chaque fenêtre
rolling_mean = df.rolling(window=window_size).mean().dropna().values/100

# Calculer la covariance des rendements pour chaque fenêtre
rolling_covariance = df.rolling.cov(window=window_size).cov().dropna().values/100

# Afficher la moyenne et la covariance pour chaque fenêtre
rolling_mean
rolling_covariance

array([[ 0.0064    ,  0.00985167,  0.00718   , ...,  0.01102667,
         0.01756333,  0.00162167],
       [ 0.00552333,  0.00691   ,  0.00469333, ...,  0.01116667,
         0.01549167, -0.00037667],
       [ 0.00475333,  0.00583333,  0.0042    , ...,  0.01052333,
         0.01548333, -0.001765  ],
       ...,
       [ 0.00495667,  0.02645333,  0.008395  , ...,  0.00644667,
         0.00554833,  0.00815333],
       [ 0.00571333,  0.02819167,  0.00906833, ...,  0.006355  ,
         0.00584833,  0.00942   ],
       [ 0.00818333,  0.03118833,  0.01158333, ...,  0.00888   ,
         0.00692333,  0.01233333]])

In [58]:
def negativeSR(w):
    R = np.sum(window.mean() * w)
    V = np.sqrt(np.dot(w.T, np.dot(window.cov(), w)))
    SR = R / V
    return -1 * SR

def optimize_portfolio(returns, risk_free_rate):
    num_assets = returns.shape[1]
    initial_weights = np.ones(num_assets) / num_assets
    bounds = [(0, 1) for _ in range(num_assets)]
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

    result = minimize(negativeSR, initial_weights, args=(returns, risk_free_rate),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x


def rolling_pf(returns, risk_free_rate, window_size):
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        weights = optimize_portfolio(window.mean(), risk_free_rate)
        pf_weights.iloc[i] = weights
    return pf_weights

In [60]:
def rolling_pf(returns, risk_free_rate, window_size):
    pf_weights = pd.DataFrame(index=returns.index, columns=returns.columns)

    for i in range(len(returns)):
        if i < window_size:
            continue
        window = returns.iloc[i-window_size:i]
        weights = optimize_portfolio(window.mean(), risk_free_rate)
        pf_weights.iloc[i] = weights
    return pf_weights

In [61]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Importer les données de rendements
# Exemple : 
returns = df

# Définir le taux sans risque et la taille de la fenêtre
risk_free_rate = 0.02
window_size = 60

# Appeler la fonction rolling_pf
portfolio_weights = rolling_pf(returns, risk_free_rate, window_size)

# Analyser les résultats
print(portfolio_weights)


TypeError: Could not convert ['   1.45   3.97   1.14  -1.24   5.20   0.82  -0.67   3.37   2.73   3.35   8.14  -1.79   4.81   2.76   5.24  -0.80   6.81   2.46   0.52  -4.17   4.39  -0.46   0.88  -4.42   1.68   6.11   1.14   0.90   6.62  -0.52   3.38  -2.62  -4.63   1.80  -3.94   5.36   3.23   4.77  -3.60 -10.64  -8.44   0.64   5.70   2.15   2.11   0.54   1.36 -11.04   3.14   1.98  -8.96  -4.85   0.18  -4.59   5.40   6.81  -1.69  -5.29 -11.01  10.73'
 '  15.55   3.68   4.80  -8.23  -0.19   9.89  -0.92   8.91   1.62   3.15   5.93  -3.00  13.09   6.98   5.67  -4.38   6.39   7.77  -3.34   1.26  27.49   1.46   5.65  -4.08   0.94   9.04   7.88   0.08   4.51  -1.22   1.30  -0.03  -1.88  -0.98 -15.05   7.01  -2.95   3.00  -8.35 -28.94 -17.31   4.90   5.95   2.44  11.37  -7.81   4.12 -22.27  12.10  -1.89 -16.73 -11.64   5.37  -1.26   8.02  17.37  -3.81  -7.87 -16.08  14.63'
 '   4.69   2.81   1.15  -3.63   4.10   3.74  -0.08   5.81   1.43   0.77   5.07  -2.77  12.07   2.89   4.76  -6.69  10.84  